In [1]:
from pyspark.sql import SparkSession
from operator import add
import pyspark
print(pyspark.__version__)


spark_session = SparkSession\
        .builder\
        .master("spark://de-i-19:7077")\
        .appName("Project_Group19")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",1)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.shuffle.service.enabled", "false")\
        .config("spark.dynamicAllocation.enabled", "false")\
        .getOrCreate()




# RDD API
spark_context = spark_session.sparkContext

# spark_context.setLogLevel("INFO")
spark_context.setLogLevel("WARN")

3.3.2


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 07:50:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#imports 
from pyspark import SparkContext, SparkConf

In [3]:
def tokenize_lines(lines):
    tokenized_lines = lines.map(lambda line: line.split('"'))
    return tokenized_lines


In [4]:
def split_tabular_data(lines):
    tabular_data = []

    for line in lines :
        csv_line = StringIO(line[0].replace('\t'))
        reader = csv.reader(csv_line, delimiter=',')
        tabular_data.extend(reader)

    # Convert the tabular data to RDD
    rdd = spark_session.sparkContext.parallelize(tabular_data)

    return rdd

In [5]:
def preprocess_data(lines):
    lowercased_lines = lines.map(lambda line: line.lower())
    tokenized_lines = tokenize_lines(lowercased_lines)
    return tokenized_lines

In [6]:
data = spark_context.textFile("hdfs://192.168.2.246:9000/user/ubuntu/amazon_reviews_us_Baby_v1_00.tsv")
print(data.take(10))

['marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date', 'US\t9970739\tR8EWA1OFT84NX\tB00GSP5D94\t329991347\tSummer Infant SwaddleMe Adjustable Infant Wrap, 3 Count\tBaby\t5\t0\t0\tN\tY\tGreat swaddled blankets\tLoved these swaddle blankets and so did my daughter, she slept so much better when tightly swaddled. Only problem was we had a very hard time getting her to sleep without it when she started rolling.\t2015-08-31', 'US\t23538442\tR2JWY4YRQD4FOP\tB00YYDDZGU\t646108902\tPacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided Design Pacifier Holder\tBaby\t5\t0\t0\tN\tN\tToo cute and really nice\tThese are adorable pacifier clips. SavvyBaby has elevated the Pacifier Holder to a new level. These are double sided strips of cotton with a loop on one end and a solid clasp on the other.<br /><br />They can be used for not only keep pacifiers off 

In [7]:
tokenized_lines = preprocess_data(data)
print("test")

test


In [8]:
mini_batch = tokenized_lines

In [9]:
mini_batch

PythonRDD[3] at RDD at PythonRDD.scala:53

In [10]:
from io import StringIO
import csv

current_rdd = mini_batch

# Collect data from the RDD
tabular_data = current_rdd.map(lambda line: line[0].split('\t'))


# Convert the tabular data to RDD
#rdd = spark_session.sparkContext.parallelize(tabular_data)

# Print the first 3 rows
# for row in tabular_data.take(3):
#     print(row)

In [11]:
from pyspark.sql.functions import corr
from pyspark.sql.functions import length

header = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category',
          'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'total_votes_float', 'helpful_votes_float']

# Assuming spark_session is your SparkSession object and rdd is your RDD
# Convert RDD to DataFrame
df = spark_session.createDataFrame(tabular_data, schema=header)
print("----Schema----")
df.printSchema()

#add review length
df = df.withColumn("review_length", length(df["review_body"]))
# as it is string now
df = df.withColumn("total_votes_float", df["total_votes"].cast("float"))
df = df.withColumn("helpful_votes_float", df["helpful_votes"].cast("float"))
df.show()


# Calculate correlation
correlation = df.corr("review_length", "helpful_votes_float")

# Print the correlation
print("Correlation between review length and helpful votes:", correlation)


IndexError: list index out of range